<a href="https://colab.research.google.com/github/MadridVP/Challenge_Telecom-X-Parte-2-Predicci-n-de-Cancelaci-n-Churn-/blob/main/Challenge_ONE_Data_Science_Telecom_X1_VM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar el archivo JSON en un DataFrame
df = pd.read_json('/content/TelecomX_Data.json')
df.head()


## **Verificar si hay problemas en los datos**


In [ ]:
df.isnull().sum()


In [ ]:
df.info()


In [ ]:
df.dtypes


In [ ]:
import pandas as pd
from pandas import json_normalize

# 1. ¿Qué columnas tienen dicts?
cols_dict = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, dict)).any()]
print("Columnas con dict:", cols_dict)

# 2. Aplanar cada una
for c in cols_dict:
    expanded = json_normalize(df[c]).add_prefix(f"{c}.")
    df = pd.concat([df.drop(columns=[c]), expanded], axis=1)

df.info()  # revisa la nueva estructura


In [ ]:
df.duplicated().sum()


In [ ]:
df.duplicated(subset=["customerID"]).sum()


In [ ]:
for col in ["Churn", "internet.plan", "phone.contract", "account.tenure"]:
    if col in df.columns:
        print(col, df[col].unique()[:20])


In [ ]:
df.shape


In [ ]:
# Revisar valores únicos de la columna Churn
df["Churn"].unique()

# Revisar valores únicos de varias columnas
for col in df.columns:
    print(f"{col}: {df[col].unique()}")


In [ ]:
df.head(3)


In [ ]:
df["Churn"] = df["Churn"].str.strip().str.title()


In [ ]:
df["account.PaperlessBilling"] = df["account.PaperlessBilling"].str.strip().str.title()
df["account.Contract"] = df["account.Contract"].str.strip().str.title()
df["account.PaymentMethod"] = df["account.PaymentMethod"].str.strip().str.title()


In [ ]:
df["Churn"].unique()
df["account.PaperlessBilling"].unique()
df["account.Contract"].unique()
df["account.PaymentMethod"].unique()


In [ ]:
print("Churn:", df["Churn"].unique())
print("PaperlessBilling:", df["account.PaperlessBilling"].unique())
print("Contract:", df["account.Contract"].unique())
print("PaymentMethod:", df["account.PaymentMethod"].unique())


In [ ]:
#Limpiar Vacios
df["Churn"] = df["Churn"].replace({"": "No"})


In [ ]:
#Unificar
df["account.Contract"] = df["account.Contract"].replace({
    "Month-To-Month": "Month-to-month",
    "One Year": "One year",
    "Two Year": "Two year"
})


In [ ]:
#Unificar mayusculas con metodo de pago
df["account.PaymentMethod"] = df["account.PaymentMethod"].replace({
    "Electronic Check": "Electronic check",
    "Mailed Check": "Mailed check",
    "Credit Card (Automatic)": "Credit card (automatic)",
    "Bank Transfer (Automatic)": "Bank transfer (automatic)"
})


In [ ]:
#Verificacion de los cambios
print("Churn:", df["Churn"].unique())
print("PaperlessBilling:", df["account.PaperlessBilling"].unique())
print("Contract:", df["account.Contract"].unique())
print("PaymentMethod:", df["account.PaymentMethod"].unique())


In [ ]:
df["Cuentas_Diarias"] = df["account.Charges.Monthly"] / 30


In [ ]:
df[["account.Charges.Monthly", "Cuentas_Diarias"]].head()


#Estandarización y transformación de datos (opcional)


In [ ]:
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
df["account.PaperlessBilling"] = df["account.PaperlessBilling"].map({"Yes": 1, "No": 0})


In [ ]:
df = df.rename(columns={
    "account.Charges.Monthly": "Cargo_Mensual",
    "account.Charges.Total": "Cargo_Total",
    "account.Contract": "Tipo_Contrato",
    "account.PaperlessBilling": "Factura_Electronica",
    "account.PaymentMethod": "Metodo_Pago",
    "Cuentas_Diarias": "Cargo_Diario"
})


In [ ]:
# Análisis descriptivo básico
print("Resumen con describe():")
print(df.describe())

# Media de cada columna numérica
print("\nMedia:")
print(df.mean(numeric_only=True))

# Mediana de cada columna numérica
print("\nMediana:")
print(df.median(numeric_only=True))

# Desviación estándar de cada columna numérica
print("\nDesviación estándar:")
print(df.std(numeric_only=True))


#Distribucion de Churn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Conteo de valores en la columna churn
conteo_churn = df['churn'].value_counts()

In [ ]:
print(df.columns)


In [ ]:
# Conteo de valores en la columna Churn
conteo_churn = df['Churn'].value_counts()

In [ ]:
# Mostrar conteo numérico
print("Conteo de Churn:")
print(conteo_churn)

In [ ]:
# Gráfico de barras
sns.countplot(x='Churn', data=df, palette='Set2')
plt.title('Distribución de evasión (Churn)')
plt.xlabel('Churn (No = se quedó, Yes = se fue)')
plt.ylabel('Cantidad de clientes')
plt.show()

In [ ]:
df['Churn'].value_counts()


In [ ]:
df['Churn_Label'] = df['Churn'].map({0: 'No se fue', 1: 'Se fue'})

In [ ]:
# Conteo
conteo_churn = df['Churn_Label'].value_counts()

In [ ]:
# Gráfico de barras
sns.countplot(x='Churn_Label', data=df, palette=['#66b3ff', '#ff9999'])
plt.title('Distribución de evasión (Churn)')
plt.xlabel('Estado del cliente')
plt.ylabel('Cantidad de clientes')
plt.show()

In [ ]:
# Crear una nueva columna con etiquetas
df['Churn_Label'] = df['Churn'].map({0: 'No se fue', 1: 'Se fue'})

# Conteo
conteo_churn = df['Churn_Label'].value_counts()

# Gráfico de barras con colores personalizados
sns.countplot(x='Churn_Label', data=df, palette={'No se fue': 'green', 'Se fue': 'orange'})
plt.title('Distribución de evasión (Churn)')
plt.xlabel('Estado del cliente')
plt.ylabel('Cantidad de clientes')
plt.show()

In [ ]:
# Gráfico de pastel con colores personalizados
plt.pie(conteo_churn, labels=conteo_churn.index, autopct='%1.1f%%', startangle=90, colors=['green', 'orange'])
plt.title('Proporción de clientes')
plt.show()

#**Churn por variables categoricas**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Analizar las Variables categóricas
variables = ['customer.gender', 'Tipo_Contrato', 'Metodo_Pago']


In [ ]:
# Crear los gráficos
for var in variables:
    plt.figure(figsize=(6,4))
    sns.countplot(x=var, hue='Churn_Label', data=df,
                  palette={'No se fue': 'green', 'Se fue': 'orange'})
    plt.title(f'Evasión según {var}')
    plt.xlabel(var)
    plt.ylabel('Cantidad de clientes')
    plt.legend(title='Churn')
    plt.xticks(rotation=15)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Analizar la Lista de variables numéricas
variables_numericas = ['Cargo_Total', 'customer.tenure']


In [ ]:
for var in variables_numericas:
    plt.figure(figsize=(6,4))
    sns.boxplot(x='Churn_Label', y=var, data=df,
                palette={'No se fue': 'green', 'Se fue': 'orange'})
    plt.title(f'{var} según estado de Churn')
    plt.xlabel('Estado del cliente')
    plt.ylabel(var)
    plt.show()

##**Informe Final**


### Introducción

El presente análisis tiene como objetivo comprender los factores que influyen en la evasión de clientes (Churn) en una empresa de telecomunicaciones.
El Churn representa a los clientes que han cancelado el servicio en un período determinado. Reducir esta tasa es crucial, ya que retener a un cliente existente es más rentable que adquirir uno nuevo.

A través de este estudio, se identificaron patrones y variables asociadas a la evasión, con el fin de proponer estrategias efectivas para mejorar la retención.

###Limpieza y Tratamiento de Datos
Importación de datos

Los datos fueron importados desde un archivo JSON provisto, utilizando pandas para su carga y manipulación.

Revisión de estructura

Se inspeccionaron las columnas, tipos de datos y valores faltantes (df.info(), df.describe()).

Creación de variables auxiliares

Se creó la columna Churn_Label para mapear los valores de Churn (0 = No se fue, 1 = Se fue) con etiquetas descriptivas.

Preparación para análisis

Se identificaron variables categóricas (género, tipo de contrato, método de pago) y numéricas (Cargo_Total, customer.tenure).[link text](https://)

##Análisis Exploratorio de Datos (EDA)

# 1. Distribución de Evasión General
El 25,7% de los clientes se fueron, mientras que el 74,3% permanecieron.

Esto indica una tasa de evasión moderada que merece atención.

Gráfico de barras y pastel mostrando proporción de Churn.

# 2. Evasión por Variables Categóricas

**Género:** No presenta diferencias significativas entre hombres y mujeres.  

**Tipo de Contrato:** Los contratos *Month-to-month* tienen la mayor tasa de evasión, mientras que *One year* y *Two year* retienen mejor a los clientes.  

**Método de Pago:** El pago mediante *Electronic check* presenta la mayor evasión; métodos automáticos muestran menor tasa.

_Gráficos comparativos por categoría._


#3. Evasión por Variables Numéricas
Cargo_Total: Los clientes que se fueron tienen un gasto total medianamente mayor, lo que podría indicar sensibilidad a costos.

Tiempo de Contrato (tenure): La evasión es más alta en clientes con menor tiempo de contrato; a mayor antigüedad, menor probabilidad de cancelación.

Boxplots para comparar distribuciones.

##**Conclusiones**

1- Tipo de contrato y método de pago son fuertes predictores de evasión.

2- Los clientes que pagan mes a mes y con Electronic check tienen mayor probabilidad de cancelar.

3- La permanencia corta está altamente asociada con la evasión.

4- Aunque el gasto total es algo mayor en los que se van, no es un factor tan determinante como el tiempo de contrato.

##**Recomendaciones Estrategicas**

* Incentivar contratos anuales o bianuales con beneficios atractivos (descuentos, upgrades).

* Promover métodos de pago automáticos para disminuir la evasión.

Programas de fidelización enfocados en clientes con menos de 12 meses de antigüedad.

* Monitoreo proactivo: Identificar clientes con perfiles de alto riesgo (contrato mensual + pago por Electronic check) y contactarlos con ofertas personalizadas.

* Evaluar estructura de precios para clientes con gasto elevado que muestran señales de deserción.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de correlación solo para variables numéricas
corr_matrix = df.corr(numeric_only=True)

# Mostrar la correlación de cada variable con Churn (ordenada)
corr_with_churn = corr_matrix['Churn'].sort_values(ascending=False)
print("Correlaciones con Churn:")
print(corr_with_churn)


In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de correlación")
plt.show()


In [ ]:
sns.boxplot(x='Churn_Label', y='Cargo_Diario', data=df,
            palette={'No se fue': 'green', 'Se fue': 'orange'})
plt.title('Relación entre Cargo Diario y Churn')
plt.show()


In [ ]:
# Lista de columnas de servicios
servicios_cols = [
    'phone.PhoneService', 'phone.MultipleLines',
    'internet.InternetService', 'internet.OnlineSecurity',
    'internet.OnlineBackup', 'internet.DeviceProtection',
    'internet.TechSupport', 'internet.StreamingTV',
    'internet.StreamingMovies'
]

# Contar cuántos servicios tiene cada cliente (No/Yes -> 0/1)
df['Servicios_Contratados'] = df[servicios_cols].apply(lambda row: sum(row == 'Yes'), axis=1)

# Gráfico
sns.boxplot(x='Churn_Label', y='Servicios_Contratados', data=df,
            palette={'No se fue': 'green', 'Se fue': 'orange'})
plt.title('Servicios contratados vs Churn')
plt.show()


##**Extracción del Archivo Tratado_Challenge Telecom X_Parte_2**


In [ ]:

df.to_csv("datos_tratados.csv", index=False)

##Eliminar Columnas Irrelevantes

In [ ]:
# Mostrar todas las columnas del dataset
print("Columnas del dataset:")
print(df.columns.tolist())


In [ ]:
df = df.drop(columns=["customerID", "Cargo_Diario"], errors="ignore")

print("Columnas finales:")
print(df.columns.tolist())


##Ecoding

In [ ]:
# Aplicar one-hot encoding a las variables categóricas
df_encoded = pd.get_dummies(df, drop_first=True)

# Revisar cómo quedó
print("Columnas después de la transformación:")
print(df_encoded.columns.tolist())
df_encoded.head()


In [ ]:
df = pd.get_dummies(df, drop_first=True)


##Verifir de la Proporción de Cancelación (Churn)

In [ ]:
# Contar cuántos clientes cancelaron y cuántos no
print(df["Churn"].value_counts())


In [ ]:
# Calcular proporción en porcentaje
print(df["Churn"].value_counts(normalize=True))


##Balancear Clases (opcional)

In [ ]:
df["Churn"].value_counts(normalize=True)


In [ ]:
!pip install imbalanced-learn


In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Definir X (features) e y (target)
X = df.drop("Churn", axis=1)
y = df["Churn"]

# Aplicar oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Revisar proporciones después
print(y_resampled.value_counts(normalize=True))


##Normalizar o Estandarizar (si es necesario)

In [ ]:
import pandas as pd
import numpy as np

# 1) QUITAR columnas que no ayudan (IDs, derivados)
df = df.drop(columns=["customerID", "Cargo_Diario"], errors="ignore")

# 2) ASEGURAR que la columna objetivo exista y sea 0/1
#    Ajusta el nombre si tu target fuera distinto a "Churn"
y = df["Churn"].copy()

# Si está como texto, lo mapeamos a 0/1
if y.dtype == "O":
    y = y.replace({
        "Yes": 1, "No": 0,
        "Se fue": 1, "No se fue": 0,
        "Canceló": 1, "Activo": 0
    })

# Por si quedan valores raros, intentamos convertir a número
y = pd.to_numeric(y, errors="coerce")
# Quitamos filas sin target válido
mask = ~y.isna()
df = df.loc[mask].reset_index(drop=True)
y  = y.loc[mask].astype(int).reset_index(drop=True)

print("Balance de clases (conteo):")
print(y.value_counts())
print("\nBalance de clases (proporción):")
print(y.value_counts(normalize=True).round(3))


In [ ]:
# Separamos features (X) de la etiqueta (y)
X = df.drop(columns=["Churn"])

# One-hot encoding para TODAS las categóricas de X
X = pd.get_dummies(X, drop_first=True)

print("Dimensiones de X e y:", X.shape, y.shape)
X.head(3)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_train.shape, X_test.shape


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

tree = DecisionTreeClassifier(
    random_state=42,  # reproducible
    # max_depth= None (por defecto). Si ves sobreajuste, prueba max_depth=4..10
)
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)
y_prob_tree = tree.predict_proba(X_test)[:, 1]

print("=== Decision Tree ===")
print("MATRIZ DE CONFUSIÓN\n", confusion_matrix(y_test, y_pred_tree))
print("\nREPORTE DE CLASIFICACIÓN\n", classification_report(y_test, y_pred_tree, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_tree).round(3))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"  # ayuda si hay desbalance
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print("=== Random Forest ===")
print("MATRIZ DE CONFUSIÓN\n", confusion_matrix(y_test, y_pred_rf))
print("\nREPORTE DE CLASIFICACIÓN\n", classification_report(y_test, y_pred_rf, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_rf).round(3))


In [ ]:
# Agregar XGBoost:
!pip install -q xgboost


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss"  # evita warnings
)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]

print("=== XGBoost ===")
print("MATRIZ DE CONFUSIÓN\n", confusion_matrix(y_test, y_pred_xgb))
print("\nREPORTE DE CLASIFICACIÓN\n", classification_report(y_test, y_pred_xgb, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_xgb).round(3))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def resumen(nombre, y_test, y_pred, y_prob):
    return {
        "modelo": nombre,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob),
    }

tabla = []
tabla.append(resumen("Decision Tree", y_test, y_pred_tree, y_prob_tree))
tabla.append(resumen("Random Forest", y_test, y_pred_rf, y_prob_rf))
tabla.append(resumen("XGBoost",       y_test, y_pred_xgb, y_prob_xgb))

pd.DataFrame(tabla).round(3).sort_values("roc_auc", ascending=False)


##Correlación y Selección de Variables

##Análisis de Correlación

In [ ]:
# Seleccionar solo las columnas numéricas
df_num = df.select_dtypes(include=["int64", "float64"])

# Mostrar las primeras columnas numéricas
print("Columnas numéricas:", df_num.columns.tolist())
df_num.head()


In [ ]:
# Calcular la matriz de correlación
corr = df_num.corr()

# Mostrar la correlación de todas las variables con Churn
print("Correlación con Churn:")
print(corr["Churn"].sort_values(ascending=False))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de correlación - Variables numéricas")
plt.show()


##Análisis Dirigido

In [ ]:
df["Churn"] = df["Churn"].map({"Yes":1, "No":0, "Se fue":1, "No se fue":0})


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.boxplot(x="Churn", y="customer.tenure", data=df)
plt.title("Tiempo de contrato vs Cancelación")
plt.xlabel("Churn (0 = No canceló, 1 = Canceló)")
plt.ylabel("Meses de contrato (tenure)")
plt.show()


In [ ]:
# Contar cuántos valores vacíos hay en cada columna
print(df.isna().sum())


In [ ]:
df["Churn"] = df["Churn"].fillna("No")


In [ ]:
df["Churn"] = df["Churn"].map({"Yes":1, "No":0, "Se fue":1, "No se fue":0})


In [ ]:
sns.boxplot(x="Churn", y="customer.tenure", data=df)


##**Modelo Predictivo**

##Separar Datos

In [ ]:
# Features (todas menos la variable target)
X = df.drop("Churn", axis=1)

# Target (lo que queremos predecir)
y = df["Churn"]


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Tamaño del dataset total:", X.shape)
print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de prueba:", X_test.shape)


##Creación de Modelos

In [ ]:
#Normalizar datos
from sklearn.preprocessing import StandardScaler

# Hacer copia para no modificar X
X_scaled = X.copy()

# Normalizar solo las variables numéricas
num_cols = X.select_dtypes(include=["int64","float64"]).columns
scaler = StandardScaler()
X_scaled[num_cols] = scaler.fit_transform(X_scaled[num_cols])


In [ ]:
# Usar Regresion Logistica
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Usar datos normalizados
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Entrenamiento
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

# Predicciones
y_pred_log = log_model.predict(X_test_scaled)

print("📊 Regresión Logística")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))


In [ ]:
print("Distribución en todo el dataset:")
print(y.value_counts())


In [ ]:
print("Distribución en entrenamiento:")
print(y_train.value_counts())

print("Distribución en prueba:")
print(y_test.value_counts())


In [ ]:
print(df["Churn"].unique()[:20])
print(y.value_counts(dropna=False))


In [ ]:
print(df["Churn"].unique())


In [ ]:
# ¿Hay columnas relacionadas con churn creadas por get_dummies?
cands = [c for c in df.columns if "churn" in c.lower()]
print(cands)


In [ ]:
# Revisamos valores únicos originales
print(df["Churn"].unique()[:20])

# Intentamos mapear a binario
df["Churn"] = df["Churn"].astype(str).str.strip()  # limpiamos espacios

df["Churn_bin"] = df["Churn"].replace({
    "Yes": 1, "No": 0,
    "Se fue": 1, "No se fue": 0,
    "True": 1, "False": 0,
    "1": 1, "0": 0
})

print("Distribución después del mapeo:")
print(df["Churn_bin"].value_counts(dropna=False))


In [ ]:
y = df["Churn_bin"].astype(int)


In [ ]:
import pandas as pd

# Cargar directamente desde GitHub
url = "https://raw.githubusercontent.com/MadridVP/Challenge_Telecom-X-Parte-2-Predicci-n-de-Cancelaci-n-Churn-/main/TelecomX_Data.json"

df = pd.read_json(url)


In [ ]:
from google.colab import files
uploaded = files.upload()

df = pd.read_json("TelecomX_Data.json")
